# ai.messages

> Define message types

In [ ]:
#| default_exp ai.messages

In [ ]:
#| hide
from nbdev.showdoc import *

In [ ]:
#| export
from dataclasses import dataclass
from typing import List, Optional, Union
from typing_extensions import Literal, TypeAlias
from pydantic import BaseModel
#
# The type definitions are adapted by reference on OPENAI specifications.
# but decide to go with dataclass instead of TypedDict
#

@dataclass
class InputAudio:
    data: str
    """Base64 encoded audio data."""

    format: Literal["wav", "mp3"]
    """The format of the encoded audio data."""

@dataclass
class PartInputAudio():
    input_audio: InputAudio

    type: Optional[Literal["input_audio"]] = "input_audio"
    """The type of the content part. Always `input_audio`."""

@dataclass
class ImageURL():
    url: str
    """Either a URL of the image or the base64 encoded image data."""

    detail: Optional[Literal["auto", "low", "high"]] = "auto"
    """Specifies the detail level of the image."""

@dataclass
class PartImage():
    image_url: ImageURL

    type: Optional[Literal["image_url"]] = "image_url"
    """The type of the content part."""

@dataclass
class PartText():
    text: str
    """The text content."""

    type: Optional[Literal["text"]] = "text"
    """The type of the content part."""


ContentPart: TypeAlias = Union[
    PartText,
    PartImage,
    PartInputAudio,
]


class UserMessage(BaseModel):
    content: Union[str, List[ContentPart]]
    """The contents of the user message."""

    role: Literal["user"] = "user"
    """The role of the messages author, in this case `user`."""

    name: Optional[str] = None
    """An optional name for the participant.

    Provides the model information to differentiate between participants of the same
    role.
    """

    def addPart(self, part: ContentPart):
        """
        Add a ContentPart to the message content. If content is a string, it will be converted to a list.
        """
        if isinstance(self.content, str):
            self.content = [PartText(text=self.content)]
        if not isinstance(self.content, list):
            self.content = []
        self.content.append(part)    


class CompletionUsage(BaseModel):
    completion_tokens: int
    """Number of tokens in the generated completion."""

    prompt_tokens: int
    """Number of tokens in the prompt."""

    total_tokens: int
    """Total number of tokens used in the request (prompt + completion)."""

    model_config = {
        "extra": "allow"
    }


class AnnotationURLCitation(BaseModel):
    end_index: int
    """The index of the last character of the URL citation in the message."""

    start_index: int
    """The index of the first character of the URL citation in the message."""

    title: str
    """The title of the web resource."""

    url: str
    """The URL of the web resource."""


class Annotation(BaseModel):
    type: Literal["url_citation"]
    """The type of the URL citation. Always `url_citation`."""

    url_citation: AnnotationURLCitation
    """A URL citation when using web search."""


class AssistantAudio(BaseModel):
    id: str
    """Unique identifier for this audio response."""

    data: str
    """
    Base64 encoded audio bytes generated by the model, in the format specified in
    the request.
    """

    expires_at: int
    """
    The Unix timestamp (in seconds) for when this audio response will no longer be
    accessible on the server for use in multi-turn conversations.
    """

    transcript: str
    """Transcript of the audio generated by the model."""


class Function(BaseModel):
    arguments: str
    """
    The arguments to call the function with, as generated by the model in JSON
    format. Note that the model does not always generate valid JSON, and may
    hallucinate parameters not defined by your function schema. Validate the
    arguments in your code before calling your function.
    """

    name: str
    """The name of the function to call."""


class ToolCallMessage(BaseModel):
    id: str
    """The ID of the tool call."""

    function: Function
    """The function that the model called."""

    type: Literal["function"]
    """The type of the tool. Currently, only `function` is supported."""


class AssistantMessage(BaseModel):
    content: Optional[str] = None
    """The contents of the message."""

    refusal: Optional[str] = None
    """The refusal message generated by the model."""

    role: Literal["assistant"] = "assistant"
    """The role of the author of this message."""

    annotations: Optional[List[Annotation]] = None
    """
    Annotations for the message, when applicable, as when using the
    """

    audio: Optional[AssistantAudio] = None
    """
    If the audio output modality is requested, this object contains data about the
    audio response from the model.
    """

    tool_calls: Optional[List[ToolCallMessage]] = None
    """The tool calls generated by the model, such as function calls."""

    model_config = {
        "extra": "allow"
    }

class Choice(BaseModel):
    finish_reason: Literal["stop", "length", "tool_calls", "content_filter", "function_call"]
    """The reason the model stopped generating tokens.

    This will be `stop` if the model hit a natural stop point or a provided stop
    sequence, `length` if the maximum number of tokens specified in the request was
    reached, `content_filter` if content was omitted due to a flag from our content
    filters, `tool_calls` if the model called a tool, or `function_call`
    (deprecated) if the model called a function.
    """

    index: int
    """The index of the choice in the list of choices."""

    message: AssistantMessage
    """A chat completion message generated by the model."""

    model_config = {
        "extra": "allow"
    }


class ChatCompletionMessage(BaseModel):
    id: str
    """A unique identifier for the chat completion."""

    choices: List[Choice]
    """A list of chat completion choices.

    Can be more than one if `n` is greater than 1.
    """

    created: int
    """The Unix timestamp (in seconds) of when the chat completion was created."""

    model: str
    """The model used for the chat completion."""

    usage: Optional[CompletionUsage] = None
    """Usage statistics for the completion request."""

    model_config = {
        "extra": "allow"
    }


In [ ]:
userMessage = UserMessage(content="Write a one-sentence bedtime story about a unicorn.")
assert userMessage.role == 'user'
assert not userMessage.name

In [ ]:
user = UserMessage(content=[
    PartText(text="What's in this image?"),
    PartImage(
        image_url=ImageURL(
            url="https://upload.wikimedia.org/wikipedia/commons/thumb/d/dd/Gfp-wisconsin-madison-the-nature-boardwalk.jpg/2560px-Gfp-wisconsin-madison-the-nature-boardwalk.jpg",
        )
    ),
])
assert user.content[0].text == "What's in this image?"
assert user.content[1].type == "image_url"

In [ ]:
user.addPart(PartText("This is a test"))
assert user.content[2].text == "This is a test"
assert user.content[2].type == "text"

In [ ]:
#| hide
import nbdev; nbdev.nbdev_export()